In [ ]:
import snowflake.connector

conn_info = dict(
    user = ...,
    role = ...,
    account = ...,
    authenticator=...,
    database=...,
    schema=...,
)
conn = snowflake.connector.connect(**conn_info)
cur = conn.cursor()


In [ ]:
from snowflake.sqlalchemy import URL
from sqlalchemy import create_engine

url = URL(**conn_info)

engine = create_engine(url)

%reload_ext sql

%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%config SqlMagic.autopandas = True

%sql select current_warehouse(), current_database(), current_schema()

In [66]:
cur.execute(
    "select '2020-12-31 00:01:02.345678 +0000'::timestamp_tz::timestamp, '2020-12-31 00:01:02.345678+00:00'::timestamp_tz, '2020-12-31 00:01:02.345678+10:00'::timestamp_tz, '2020-12-31 00:01:02.345678+10:00'::timestamp_tz::timestamp"
).fetchall()

[(datetime.datetime(2020, 12, 31, 0, 1, 2, 345678),
  datetime.datetime(2020, 12, 31, 0, 1, 2, 345678, tzinfo=<UTC>),
  datetime.datetime(2020, 12, 31, 0, 1, 2, 345678, tzinfo=pytz.FixedOffset(600)),
  datetime.datetime(2020, 12, 31, 0, 1, 2, 345678))]

In [72]:
%sql select '2020-12-31 00:01:02.345678+00:00'::timestamp_tz::timestamp, '2020-12-31 00:01:02.345678+00:00'::timestamp_tz, '2020-12-31 00:01:02.345678+10:00'::timestamp_tz, '2020-12-31 00:01:02.345678 +1000'::timestamp_tz

,'2020-12-31 00:01:02.345678+00:00' ::TIMESTAMP_TZ::TIMESTAMP,'2020-12-31 00:01:02.345678+00:00' ::TIMESTAMP_TZ,'2020-12-31 00:01:02.345678+10:00' ::TIMESTAMP_TZ,'2020-12-31 00:01:02.345678 +1000' ::TIMESTAMP_TZ
0,2020-12-31 00:01:02.345678,2020-12-31 00:01:02.345678+00:00,2020-12-31 00:01:02.345678+10:00,2020-12-31 00:01:02.345678+10:00


In [76]:
cur.execute(
    "select '2020-12-31 00:01:02.345678+10:00'::timestamp_tz::timestamp, '2020-12-31 00:01:02.345678'::timestamp_tz::timestamp"
).fetchall()

[(datetime.datetime(2020, 12, 31, 0, 1, 2, 345678),
  datetime.datetime(2020, 12, 31, 0, 1, 2, 345678))]

In [80]:
import datetime

import pandas as pd
import pytz
import snowflake.connector.pandas_tools

cur.execute("create or replace table example (UPDATE_AT_NTZ timestamp_ntz(9))")
# cur.execute("create table example (UPDATE_AT_NTZ timestamp)")

now_utc = datetime.datetime.now(pytz.utc)
df = pd.DataFrame([(now_utc,)], columns=["UPDATE_AT_NTZ"])
snowflake.connector.pandas_tools.write_pandas(conn, df, "EXAMPLE")

cur.execute("select * from example")

assert cur.fetchall() == [(now_utc.replace(tzinfo=None),)]

In [57]:
%sql select '2020-12-31 00:01:02.345678 +0000'::timestamp_tz::timestamp, '2020-12-31 00:01:02.345678 +0000'::timestamp_tz

INFO:backoff:Backing off send_request(...) for 0.8s (requests.exceptions.ReadTimeout: HTTPSConnectionPool(host='app.posthog.com', port=443): Read timed out. (read timeout=15))


,'2020-12-31 00:01:02.345678 +0000' ::TIMESTAMP_TZ::TIMESTAMP,'2020-12-31 00:01:02.345678 +0000' ::TIMESTAMP_TZ
0,2020-12-31 00:01:02.345678,2020-12-31 00:01:02.345678+00:00


In [22]:
%sql SELECT system$typeof(to_date(to_timestamp(0)))

,SYSTEM$TYPEOF(TO_DATE(TO_TIMESTAMP(0)))
0,DATE[SB4]


In [14]:
cur.execute("SELECT to_timestamp('2013-04-05 01:02:03')").fetchall()

[(datetime.datetime(2013, 4, 5, 1, 2, 3),)]